In [12]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D, GlobalAveragePooling2D
from keras import regularizers
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import pickle
from sklearn.metrics import confusion_matrix
from PyFunctions import Functions as func
from PyFunctions import Viz
from keras.applications.vgg16 import VGG16
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

In [4]:
x_train, x_test, y_train, y_test = func.get_emotion_splits(dim = (224,224), model_type = 'vgg16') 
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

TRAIN Value Count
1    6000
2    4965
3    4830
0    3995
dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
TEST Value Count
1    1774
3    1247
2    1233
0     958
dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(19790, 224, 224, 3) (5212, 224, 224, 3)
(19790, 4) (5212, 4)


In [10]:
def get_vgg16(dim):
    model = Sequential()
    baseModel = VGG16(weights="imagenet", include_top=False,
        input_tensor=Input(shape=dim), classes = 4)
    
    # loop over all layers in the base model and freeze them so they will
    # *not* be updated during the first training process
    for layer in baseModel.layers:
        layer.trainable = False
        
    model.add(baseModel)
    model.add(GlobalAveragePooling2D())
#     model.add(Flatten(name="flatten"))
    model.add(Dense(1024, activation="relu"))
    model.add(Dense(512, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4, activation="softmax", name = 'Output'))
 
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [13]:
early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=8, min_delta = .00075)
model_checkpoint = ModelCheckpoint(f'ModelWeights/VGG16_Emotions.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')
lr_plat = ReduceLROnPlateau(patience = 5, mode = 'min')
epochs = 2000
batch_size = 64
    

dim = (x_train.shape[1], x_train.shape[2], x_train.shape[3])
vgg16 = get_vgg16(dim =dim)
vgg16 = load_model('ModelWeights/VGG16_Emotions.h5')
    
augmentation =ImageDataGenerator(rotation_range = 20, width_shift_range = .2, height_shift_range = .2, 
                                                       horizontal_flip = True, shear_range = .15, 
                                 fill_mode = 'nearest', zoom_range = .15)
augmentation.fit(x_train)
vgg16_history = vgg16.fit_generator(augmentation.flow(x_train, y_train, batch_size = batch_size),
            epochs = epochs, 
     callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)

Epoch 1/2000
310/310 [==============================] - 584s 2s/step - loss: 1.2218 - acc: 0.4602 - val_loss: 1.2138 - val_acc: 0.4605

Epoch 00001: val_loss improved from inf to 1.21382, saving model to ModelWeights/VGG16_Emotions.h5
Epoch 2/2000
310/310 [==============================] - 582s 2s/step - loss: 1.2142 - acc: 0.4600 - val_loss: 1.2232 - val_acc: 0.4526

Epoch 00002: val_loss did not improve from 1.21382
Epoch 3/2000
310/310 [==============================] - 584s 2s/step - loss: 1.1984 - acc: 0.4742 - val_loss: 1.2059 - val_acc: 0.4705

Epoch 00003: val_loss improved from 1.21382 to 1.20587, saving model to ModelWeights/VGG16_Emotions.h5
Epoch 4/2000
310/310 [==============================] - 584s 2s/step - loss: 1.1929 - acc: 0.4770 - val_loss: 1.2150 - val_acc: 0.4637

Epoch 00004: val_loss did not improve from 1.20587
Epoch 5/2000
310/310 [==============================] - 581s 2s/step - loss: 1.1867 - acc: 0.4788 - val_loss: 1.2104 - val_acc: 0.4547

Epoch 00005: val